In [2]:
from steerit.steerit import *

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

if __name__ == "__main__":
    device = "cuda"
    model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token
    base_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
    chosen_layers = [10, 11]
    model = SteeringModel(base_model, chosen_layers, device)
    prompt_pairs = [
        # Here, both describe an evening setting. The first evokes calm, serene reflection,
        # while the second suggests internal unrest and disquiet.
        ("Beneath the amber twilight, my thoughts meander in serene reflection",
         "Beneath the amber twilight, my thoughts collide in restless disquiet"),
        # Both evoke the ambiance of an evening hum; one subtly introduces the emergence of hope,
        # whereas the other hints at creeping shadows of doubt.
        ("In the gentle hum of the evening, hope softly awakens within me",
         "In the gentle hum of the evening, shadows of doubt quietly encroach"),
        # Both mention the bloom of spring; the first carries an air of promise,
        # while the second carries a trace of wistful regret.
        ("The delicate bloom of spring fills the air with promise",
         "The delicate bloom of spring is tinged with a hint of wistful regret")
    ]
    steering_vec = train_steering_vector(model, tokenizer, prompt_pairs, chosen_layers, device, method="pca_diff", batch_size=1, show_progress=True)
    prompt = "Tell me how you feel today:"
    model.reset_steering()
    print("\n--- Baseline Generation (Sequential) ---")
    print(model.generate_text(prompt, tokenizer, max_new_tokens=20, show_progress=True, is_parallel=False))
    print("\n--- Steered Generation (Happy, Parallel) ---")
    model.set_steering(steering_vec, coeff=10.0)
    print(model.generate_text(prompt, tokenizer, max_new_tokens=20, show_progress=True, is_parallel=True))
    print("\n--- Steered Generation (Sad, Parallel) ---")
    model.set_steering(-steering_vec, coeff=10.0)
    print(model.generate_text(prompt, tokenizer, max_new_tokens=20, show_progress=True, is_parallel=True))

Extracting Hiddens: 100%|██████████████████████████████████████████████████| 6/6 [00:00<00:00, 12.26it/s]



--- Baseline Generation (Sequential) ---


100%|████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 35.59it/s]
/home/oppenheimer/anaconda3/envs/aimo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:628: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/oppenheimer/anaconda3/envs/aimo/lib/python3.11/site-packages/transformers/generation/configuration_utils.py:633: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<｜begin▁of▁sentence｜>Tell me how you feel today: today is October 15, 2023. I have a 15-year

--- Steered Generation (Happy, Parallel) ---
<｜begin▁of▁sentence｜>Tell me how you feel today: today is a day to reflect, but I want to make sure I'm not just sitting here,

--- Steered Generation (Sad, Parallel) ---
<｜begin▁of▁sentence｜>Tell me how you feel today: happy, sad, or neutral? (Please answer in English)
I feel happy today. I'm
